## Using David Hogg's Cross-validation approach

In [1]:
from Chempy.score_function import Hogg_scoring
%timeit -r 1 -n 1 Hogg_scoring()

Running MCMC iteration 1 of 22
['Proto-sun']
first minimization for each star separately took:  1 seconds


Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/home/philcox/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/philcox/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/philcox/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/philcox/anaconda3/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 481, in posterior_function_returning_predictions
    posterior, abundance_list, element_list = posterior_function_predictions(changing_parameter,a)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 506, in posterior_function_predictions
    abundance_list,elements_to_trace = cem_real2(a)
  File "/home/philcox/Chempy/Chempy/cem_function.py", line 336, in cem_real2
 

KeyboardInterrupt: 

In [9]:
import fileinput
import sys
for index in [1., 1.2, 1.4, 1.7, 2., 2.5, 3., 4., 5., 6., 8., 10., 12., 15., 18., 22., 26., 30., 40., 50.]:
    for line in fileinput.input("Chempy/parameter.py", inplace=True):
        if "\tbeta_param =" in line:
            print("\tbeta_param = %.2f" %(index))
        else:
            print(line,end='')
    del sys.modules['Chempy.parameter']
    Hogg_scoring(index)

TypeError: Hogg_scoring() takes 0 positional arguments but 1 was given

In [ ]:
## Code to rewrite parameter file for each element in turn, so as to run MCMC for 21/22 elements only
# This is definitely not a good implementation (involves rewriting entire parameter file),
# But other steps are far slower

# Starting elements (copied from original parameter file)
elements_to_trace = ['Al', 'Ar', 'B', 'Be', 'C', 'Ca', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'K', 'Li', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Sc', 'Si', 'Ti', 'V', 'Zn']
orig = "\telements_to_trace = "+str(elements_to_trace) # Original element string

# Calculate required Chempy elements
sol_dat = np.load("Chempy/input/stars/Proto-sun.npy")
el_names = []
for el in elements_to_trace:
    if el in sol_dat.dtype.names:
        el_names.append(el)
np.save("Hogg/elements.npy",np.array(el_names))
        
# Create new parameter names
newstr = []
for i,el in enumerate(el_names):
    if el !='Zn':
        newstr.append(orig.replace("'"+str(el)+"', ",""))
    else:
        newstr.append(orig.replace("'"+str(el)+"', ",""))
        
        
for i in range(2): #len(el_names)): # Iterate over removed element
    for line in fileinput.input("Chempy/parameter.py", inplace=True):
        if "\telements_to_trace" in line:
            print(newstr[i])
        else:
            print(line,end='')
    del sys.modules['Chempy.parameter']
    from Chempy.parameter import ModelParameters
    a = ModelParameters()
    ##############
    # MCMC using 21 elements only goes here
    print('Running MCMC iteration %d of %d' %(i+1,len(el_names)))
    multi_star_optimization()
    restructure_chain('mcmc/')
    positions = np.load('mcmc/posteriorPDF.npy')
    median = []
    up = []
    low = []
    for j in range(len(a.p0)):
        median.append(np.percentile(positions[:,j],50))
        low.append(np.percentile(positions[:,j],15.865))
        up.append(np.percentile(positions[:,j],100-15.865))
    np.save('Hogg/median%d.npy' %(i),np.array(median))
    np.save('Hogg/low%d.npy' %(i),np.array(low))
    np.save('Hogg/up%d.npy' %(i),np.array(up))
    ##############

In [2]:
index = 3
directory = "Scores/Hogg_"+str(index)+"/"

In [5]:
directory
import os

In [6]:
if not os.path.exists(directory):
    os.makedirs(directory)